In [1]:
import numpy as np
#import cv2
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [2]:
import tensorflow as tf
from tensorflow import keras
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.test.is_built_with_cuda()
print(tf.version.VERSION)

Num GPUs Available:  1
2.8.0


In [3]:
dir_path = 'C:/Users/HP/Desktop/smart_bin_code/data/archive/Garbage classification/Garbage classification'
print(dir_path)

C:/Users/HP/Desktop/smart_bin_code/data/archive/Garbage classification/Garbage classification


In [4]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [5]:
len(img_list)

2527

In [6]:
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

train_generator=train.flow_from_directory(dir_path,
                                          target_size=(300,300),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

test_generator=test.flow_from_directory(dir_path,
                                        target_size=(300,300),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

labels = (train_generator.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [7]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 300, 300, 3), (32, 6))

In [8]:
print (train_generator.class_indices)

Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(Labels)

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [9]:
model=Sequential()
#Convolution blocks

model.add(Conv2D(32,(3,3), padding='same',input_shape=(300,300,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
#model.add(SpatialDropout2D(0.5)) # No accuracy

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
#model.add(SpatialDropout2D(0.5))

model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 

#Classification layers
model.add(Flatten())

model.add(Dense(64,activation='relu'))
#model.add(SpatialDropout2D(0.5))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

filepath="trained_model.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]



In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43808)             0

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc']) # RMS PROP - No accuracy

#es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=500,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 

C:\Users\HP\anaconda3\envs\training_env\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/500
71/71 [==============================] - 47s 421ms/step - loss: 1.7494 - acc: 0.2094 - val_loss: 1.6530 - val_acc: 0.2411

Epoch 00001: val_acc improved from -inf to 0.24107, saving model to trained_model.h5
Epoch 2/500
71/71 [==============================] - 20s 258ms/step - loss: 1.6304 - acc: 0.2906 - val_loss: 1.5691 - val_acc: 0.3304

Epoch 00002: val_acc improved from 0.24107 to 0.33036, saving model to trained_model.h5
Epoch 3/500
71/71 [==============================] - 19s 248ms/step - loss: 1.5923 - acc: 0.3048 - val_loss: 1.5374 - val_acc: 0.3839

Epoch 00003: val_acc improved from 0.33036 to 0.38393, saving model to trained_model.h5
Epoch 4/500
71/71 [==============================] - 19s 253ms/step - loss: 1.4706 - acc: 0.3824 - val_loss: 1.3243 - val_acc: 0.5000

Epoch 00004: val_acc improved from 0.38393 to 0.50000, saving model to trained_model.h5
Epoch 5/500
71/71 [==============================] - 19s 260ms/step - loss: 1.4123 - acc: 0.4251 - val_loss: 1.

71/71 [==============================] - 19s 261ms/step - loss: 0.7459 - acc: 0.7424 - val_loss: 0.7907 - val_acc: 0.7321

Epoch 00042: val_acc did not improve from 0.75893
Epoch 43/500
71/71 [==============================] - 20s 263ms/step - loss: 0.7530 - acc: 0.7286 - val_loss: 0.7654 - val_acc: 0.7500

Epoch 00043: val_acc did not improve from 0.75893
Epoch 44/500
71/71 [==============================] - 20s 281ms/step - loss: 0.7145 - acc: 0.7398 - val_loss: 0.7984 - val_acc: 0.7009

Epoch 00044: val_acc did not improve from 0.75893
Epoch 45/500
71/71 [==============================] - 20s 264ms/step - loss: 0.7312 - acc: 0.7389 - val_loss: 0.7856 - val_acc: 0.7277

Epoch 00045: val_acc did not improve from 0.75893
Epoch 46/500
71/71 [==============================] - 19s 262ms/step - loss: 0.7368 - acc: 0.7344 - val_loss: 0.7726 - val_acc: 0.7321

Epoch 00046: val_acc did not improve from 0.75893
Epoch 47/500
71/71 [==============================] - 20s 264ms/step - loss: 0.6920


Epoch 00085: val_acc did not improve from 0.81696
Epoch 86/500
71/71 [==============================] - 19s 262ms/step - loss: 0.5072 - acc: 0.8298 - val_loss: 0.7357 - val_acc: 0.7679

Epoch 00086: val_acc did not improve from 0.81696
Epoch 87/500
71/71 [==============================] - 20s 266ms/step - loss: 0.5448 - acc: 0.8164 - val_loss: 0.6614 - val_acc: 0.7723

Epoch 00087: val_acc did not improve from 0.81696
Epoch 88/500
71/71 [==============================] - 20s 264ms/step - loss: 0.4846 - acc: 0.8338 - val_loss: 0.5608 - val_acc: 0.7991

Epoch 00088: val_acc did not improve from 0.81696
Epoch 89/500
71/71 [==============================] - 20s 266ms/step - loss: 0.4927 - acc: 0.8262 - val_loss: 0.6533 - val_acc: 0.7812

Epoch 00089: val_acc did not improve from 0.81696
Epoch 90/500
71/71 [==============================] - 20s 264ms/step - loss: 0.5017 - acc: 0.8262 - val_loss: 0.7186 - val_acc: 0.7589

Epoch 00090: val_acc did not improve from 0.81696
Epoch 91/500
71/71 


Epoch 00129: val_acc did not improve from 0.81696
Epoch 130/500
71/71 [==============================] - 20s 266ms/step - loss: 0.3940 - acc: 0.8614 - val_loss: 0.8716 - val_acc: 0.7634

Epoch 00130: val_acc did not improve from 0.81696
Epoch 131/500
71/71 [==============================] - 20s 266ms/step - loss: 0.5037 - acc: 0.8289 - val_loss: 0.8004 - val_acc: 0.7455

Epoch 00131: val_acc did not improve from 0.81696
Epoch 132/500
71/71 [==============================] - 19s 257ms/step - loss: 0.5099 - acc: 0.8226 - val_loss: 0.7252 - val_acc: 0.7723

Epoch 00132: val_acc did not improve from 0.81696
Epoch 133/500
71/71 [==============================] - 20s 262ms/step - loss: 0.4546 - acc: 0.8494 - val_loss: 0.8545 - val_acc: 0.7634

Epoch 00133: val_acc did not improve from 0.81696
Epoch 134/500
71/71 [==============================] - 20s 266ms/step - loss: 0.4271 - acc: 0.8498 - val_loss: 0.8073 - val_acc: 0.7812

Epoch 00134: val_acc did not improve from 0.81696
Epoch 135/500



Epoch 00173: val_acc did not improve from 0.81696
Epoch 174/500
71/71 [==============================] - 19s 259ms/step - loss: 0.3323 - acc: 0.8877 - val_loss: 0.8321 - val_acc: 0.7321

Epoch 00174: val_acc did not improve from 0.81696
Epoch 175/500
71/71 [==============================] - 19s 257ms/step - loss: 0.3112 - acc: 0.8913 - val_loss: 0.9900 - val_acc: 0.7411

Epoch 00175: val_acc did not improve from 0.81696
Epoch 176/500
71/71 [==============================] - 19s 258ms/step - loss: 0.3060 - acc: 0.8899 - val_loss: 1.0497 - val_acc: 0.7723

Epoch 00176: val_acc did not improve from 0.81696
Epoch 177/500
71/71 [==============================] - 19s 256ms/step - loss: 0.2922 - acc: 0.8966 - val_loss: 0.9904 - val_acc: 0.7679

Epoch 00177: val_acc did not improve from 0.81696
Epoch 178/500
71/71 [==============================] - 19s 256ms/step - loss: 0.3419 - acc: 0.8846 - val_loss: 0.8845 - val_acc: 0.7500

Epoch 00178: val_acc did not improve from 0.81696
Epoch 179/500
